Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import math as math
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
train_subset = 10000
l2_penalty = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
 
  logits = tf.matmul(tf_train_dataset,weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + l2_penalty * tf.nn.l2_loss(weights)
  # TODO add L2 regularization to the unregu

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
num_steps = 901

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 47.349861
Training accuracy: 9.4%
Validation accuracy: 11.8%
Loss at step 100: 11.959171
Training accuracy: 74.0%
Validation accuracy: 73.5%
Loss at step 200: 4.528694
Training accuracy: 79.0%
Validation accuracy: 77.5%
Loss at step 300: 1.997958
Training accuracy: 82.4%
Validation accuracy: 80.4%
Loss at step 400: 1.138931
Training accuracy: 83.5%
Validation accuracy: 81.6%
Loss at step 500: 0.844125
Training accuracy: 83.8%
Validation accuracy: 82.0%
Loss at step 600: 0.741702
Training accuracy: 84.0%
Validation accuracy: 82.4%
Loss at step 700: 0.705771
Training accuracy: 84.1%
Validation accuracy: 82.4%
Loss at step 800: 0.693067
Training accuracy: 84.1%
Validation accuracy: 82.5%
Loss at step 900: 0.688543
Training accuracy: 84.0%
Validation accuracy: 82.5%
Test accuracy: 88.4%


In [7]:
batch_size = 10
num_nodes = 1024

def runHiddenNetwork(batch_size=128, num_steps = 3001, use_l2 = True, use_dropout = False, prob_val = 1.0) :
    graphRelu = tf.Graph()
    with graphRelu.as_default():
        keep_prob = tf.placeholder(tf.float32)
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        # Variables.
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
        biases_1 = tf.Variable(tf.zeros([num_nodes]))
        weights_2 = tf.Variable( tf.truncated_normal([num_nodes, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
      
        relu_layer = tf.nn.relu(logits_1)  
        if(use_dropout) :
            relu_layer = tf.nn.dropout(relu_layer,keep_prob)
        logits_2 = tf.matmul(relu_layer, weights_2) + biases_2 # out layer --------------------------------
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_2,labels=tf_train_labels))
        if(use_l2):
            loss = loss + l2_penalty * tf.nn.l2_loss(weights_1)  + l2_penalty * tf.nn.l2_loss(weights_2)
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        train_prediction = tf.nn.softmax(logits_2)
        
        logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
        relu_layer = tf.nn.relu(logits_1)
        logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
        test_prediction =  tf.nn.softmax(logits_2)       
        
        with tf.Session(graph=graphRelu) as session:
            tf.initialize_all_variables().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:prob_val}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                if (step % 500 == 0):
                    print("Minibatch loss at step {}: {}".format(step, l))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))
            #test_prediction.eval()

In [8]:
runHiddenNetwork(batch_size=128, use_l2 = True)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3445.111083984375
Minibatch accuracy: 16.4
Minibatch loss at step 500: 21.402061462402344
Minibatch accuracy: 82.8
Minibatch loss at step 1000: 0.8653470277786255
Minibatch accuracy: 84.4
Minibatch loss at step 1500: 0.6557348370552063
Minibatch accuracy: 84.4
Minibatch loss at step 2000: 0.864836573600769
Minibatch accuracy: 79.7
Minibatch loss at step 2500: 0.8298549056053162
Minibatch accuracy: 79.7
Minibatch loss at step 3000: 0.8453985452651978
Minibatch accuracy: 79.7
Test accuracy: 90.2


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# TODO add cell for 
runHiddenNetwork(batch_size=15, use_l2 = True,prob_val=0.3)

Initialized
Minibatch loss at step 0: 3425.365478515625
Minibatch accuracy: 13.3
Minibatch loss at step 500: 25.383543014526367
Minibatch accuracy: 60.0
Minibatch loss at step 1000: 1.0740028619766235
Minibatch accuracy: 73.3
Minibatch loss at step 1500: 0.9846184849739075
Minibatch accuracy: 80.0
Minibatch loss at step 2000: 2.09094500541687
Minibatch accuracy: 73.3
Minibatch loss at step 2500: 0.9718583226203918
Minibatch accuracy: 80.0
Minibatch loss at step 3000: 1.0301719903945923
Minibatch accuracy: 80.0
Test accuracy: 80.5


In [10]:
runHiddenNetwork(batch_size=15, use_l2 = False)

Initialized
Minibatch loss at step 0: 322.5880126953125
Minibatch accuracy: 6.7
Minibatch loss at step 500: 858.4832763671875
Minibatch accuracy: 53.3
Minibatch loss at step 1000: 37.38526153564453
Minibatch accuracy: 53.3
Minibatch loss at step 1500: 16.944679260253906
Minibatch accuracy: 73.3
Minibatch loss at step 2000: 218.5354461669922
Minibatch accuracy: 46.7
Minibatch loss at step 2500: 13.278840065002441
Minibatch accuracy: 60.0
Minibatch loss at step 3000: 2.4714014530181885
Minibatch accuracy: 60.0
Test accuracy: 60.0


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
runHiddenNetwork(batch_size=128, use_l2 = True, use_dropout=True, prob_val=1)

Initialized
Minibatch loss at step 0: 3424.615966796875
Minibatch accuracy: 14.1
Minibatch loss at step 500: 21.43058967590332
Minibatch accuracy: 81.2
Minibatch loss at step 1000: 0.859840452671051
Minibatch accuracy: 85.2
Minibatch loss at step 1500: 0.6387560367584229
Minibatch accuracy: 85.2
Minibatch loss at step 2000: 0.8508397936820984
Minibatch accuracy: 80.5
Minibatch loss at step 2500: 0.8196816444396973
Minibatch accuracy: 80.5
Minibatch loss at step 3000: 0.8435727953910828
Minibatch accuracy: 79.7
Test accuracy: 90.1


In [12]:
runHiddenNetwork(batch_size=128, use_l2 = False, use_dropout=False)

Initialized
Minibatch loss at step 0: 306.6317138671875
Minibatch accuracy: 10.9
Minibatch loss at step 500: 21.819091796875
Minibatch accuracy: 75.0
Minibatch loss at step 1000: 13.25208854675293
Minibatch accuracy: 82.0
Minibatch loss at step 1500: 7.70543098449707
Minibatch accuracy: 82.0
Minibatch loss at step 2000: 3.63161301612854
Minibatch accuracy: 77.3
Minibatch loss at step 2500: 3.66038179397583
Minibatch accuracy: 81.2
Minibatch loss at step 3000: 3.9912567138671875
Minibatch accuracy: 73.4
Test accuracy: 88.2


In [13]:
runHiddenNetwork(batch_size=128, use_l2 = True, use_dropout=False)

Initialized
Minibatch loss at step 0: 3405.5517578125
Minibatch accuracy: 8.6
Minibatch loss at step 500: 21.45124053955078
Minibatch accuracy: 81.2
Minibatch loss at step 1000: 0.8566844463348389
Minibatch accuracy: 85.2
Minibatch loss at step 1500: 0.6463170647621155
Minibatch accuracy: 84.4
Minibatch loss at step 2000: 0.8552833199501038
Minibatch accuracy: 79.7
Minibatch loss at step 2500: 0.8254164457321167
Minibatch accuracy: 80.5
Minibatch loss at step 3000: 0.8451869487762451
Minibatch accuracy: 79.7
Test accuracy: 90.3


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [78]:
batch_size = 10

l2_penalty = 0.01

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))

def runHiddenNetworkMulti(batch_size=128, num_steps = 3001) :
    graphRelu = tf.Graph()
    with graphRelu.as_default():
        keep_prob = tf.placeholder(tf.float32)
        global_step = tf.Variable(0)
        decay_steps = tf.Variable(0)

        decay_rate =  tf.constant(0.96)
        learning_rate = tf.train.exponential_decay(
            learning_rate=0.5,global_step=global_step,decay_rate=decay_rate,decay_steps = decay_steps
        ) #number of steps taken.
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        
        
        # Variables.
        
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1],
                                               stddev=math.sqrt(2.0/(image_size*image_size))))
        
        weights_test = tf.truncated_normal([image_size * image_size, hidden_nodes_1])
        
        print(weights_1)
        print(weights_test)

        biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
        weights_2 = tf.Variable( tf.truncated_normal([hidden_nodes_1, hidden_nodes_2],
                                                    stddev=math.sqrt(2.0/(hidden_nodes_1))))
        biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
        weights_3 = tf.Variable( tf.truncated_normal([hidden_nodes_2, hidden_nodes_3],
                                                    stddev=math.sqrt(2.0/(hidden_nodes_2))))
        biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))

        weights_4 = tf.Variable( tf.truncated_normal([hidden_nodes_3, num_labels],
                                                    stddev=math.sqrt(2.0/(hidden_nodes_3))))
        biases_4 = tf.Variable(tf.zeros([num_labels]))

        logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
        relu_layer_1 = tf.nn.dropout(tf.nn.relu(logits_1),keep_prob)
        logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2     
        relu_layer_2 = tf.nn.dropout(tf.nn.relu(logits_2),keep_prob)
        logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
        relu_layer_3 = tf.nn.dropout(tf.nn.relu(logits_3),keep_prob)
        logits_4 = tf.matmul(relu_layer_3, weights_4) + biases_4
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_4,labels=tf_train_labels))
            
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        train_prediction = tf.nn.softmax(logits_4)
        
        # Predictions for test
        logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
        relu_layer_1 = tf.nn.relu(logits_1)
        logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
        relu_layer_2 = tf.nn.relu(logits_2)
        logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
        relu_layer_3 = tf.nn.relu(logits_3)
        logits_4 = tf.matmul(relu_layer_3, weights_4) + biases_4
        test_prediction =  tf.nn.softmax(logits_4)   

        
        with tf.Session(graph=graphRelu) as session:
            tf.initialize_all_variables().run()
            print("Initialized")
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:1.0, global_step:step,decay_steps:num_steps}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                if (step % 500 == 0):
                    print("Minibatch loss at step {}: {}".format(step, l))
                    print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))


In [79]:
runHiddenNetworkMulti(batch_size=128)

<tf.Variable 'Variable_2:0' shape=(784, 1024) dtype=float32_ref>
Tensor("truncated_normal_1:0", shape=(784, 1024), dtype=float32)
Initialized
Minibatch loss at step 0: 2.3900697231292725
Minibatch accuracy: 12.5
Minibatch loss at step 500: 0.5901281833648682
Minibatch accuracy: 85.9
Minibatch loss at step 1000: 0.3898574113845825
Minibatch accuracy: 88.3
Minibatch loss at step 1500: 0.2717112898826599
Minibatch accuracy: 94.5
Minibatch loss at step 2000: 0.35647156834602356
Minibatch accuracy: 88.3
Minibatch loss at step 2500: 0.43629390001296997
Minibatch accuracy: 85.9
Minibatch loss at step 3000: 0.41051414608955383
Minibatch accuracy: 87.5
Test accuracy: 94.6
